# Data-Driven Business Insights & Predictive Modeling – Capstone

**Author:** Shruti Arvind Kherade  
**Dataset:** Synthetic unified dataset (customers, products, transactions, subscriptions)  
**Tools:** Python, Pandas, NumPy, Scikit-learn, Matplotlib, Seaborn, SciPy

This end-to-end notebook consolidates topics from multiple DSBA projects into one comprehensive workflow: EDA, cleaning, feature engineering, hypothesis testing, classification, regression, clustering, model evaluation, and business recommendations.


In [ ]:

# %%capture
import os, warnings, math
warnings.filterwarnings('ignore')

import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (accuracy_score, roc_auc_score, classification_report,
                             r2_score, mean_squared_error, silhouette_score)
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.cluster import KMeans
from scipy import stats

# Load data
base_dir = '/mnt/data'
paths = {
    'merged': os.path.join(base_dir, 'capstone_unified_dataset.csv'),
    'customers': os.path.join(base_dir, 'capstone_customers.csv'),
    'products': os.path.join(base_dir, 'capstone_products.csv'),
    'transactions': os.path.join(base_dir, 'capstone_transactions.csv'),
    'subscriptions': os.path.join(base_dir, 'capstone_subscriptions.csv'),
}
for p in paths.values():
    assert os.path.exists(p), f'Missing dataset file: {p}'

df = pd.read_csv(paths['merged'])
customers = pd.read_csv(paths['customers'])
products = pd.read_csv(paths['products'])
transactions = pd.read_csv(paths['transactions'])
subscriptions = pd.read_csv(paths['subscriptions'])

df['order_date'] = pd.to_datetime(df['order_date'])
customers['signup_date'] = pd.to_datetime(customers['signup_date'])
subscriptions['start_date'] = pd.to_datetime(subscriptions['start_date'])
subscriptions['end_date'] = pd.to_datetime(subscriptions['end_date'])

print('Rows:', len(df), '| Columns:', df.shape[1])
df.head(3)


## Contents
1) EDA
2) Visualizations
3) Cleaning & Feature Engineering
4) Hypothesis Testing
5) Classification (Churn)
6) Regression (CLV)
7) Clustering
8) Time-Series Trend
9) Insights & Recommendations

In [ ]:

# ## 1) EDA
df.describe(include='all').T.head(20)


In [ ]:

# ### Visualizations
plt.figure()
cat_rev = df.groupby('category')['revenue'].sum().sort_values(ascending=False).head(10)
cat_rev.plot(kind='bar')
plt.title('Revenue by Category')
plt.xlabel('Category'); plt.ylabel('Revenue')
plt.show()

plt.figure()
monthly = df.set_index('order_date').resample('M')['revenue'].sum()
monthly.plot()
plt.title('Monthly Revenue Trend')
plt.xlabel('Month'); plt.ylabel('Revenue')
plt.show()

plt.figure()
sns.histplot(df['age'], bins=30, kde=True)
plt.title('Customer Age Distribution')
plt.show()


In [ ]:

# ## 2) Cleaning & Feature Engineering
df['revenue_cap'] = np.clip(df['revenue'], 0, df['revenue'].quantile(0.99))

cust_features = df.groupby('customer_id').agg({
    'order_date': lambda x: (df['order_date'].max() - x.max()).days,
    'order_id': 'nunique',
    'revenue': 'sum',
    'quantity': 'sum',
    'discount': 'mean',
    'age': 'mean'
}).rename(columns={'order_date':'recency_days','order_id':'frequency','revenue':'monetary','age':'avg_age'}).reset_index()

cust_full = (cust_features
             .merge(customers[['customer_id','country','region','segment','primary_channel','is_student','has_membership','churn_risk']], on='customer_id', how='left'))
cust_full.head()


In [ ]:

# ## 3) Hypothesis Testing
members = cust_full.loc[cust_full['has_membership']==1, 'monetary']
non_members = cust_full.loc[cust_full['has_membership']==0, 'monetary']
t_stat, p_val = stats.ttest_ind(members, non_members, equal_var=False)
print('T-test (members vs non-members monetary): t=%.3f, p=%.6f' % (t_stat, p_val))

groups = [cust_full.loc[cust_full['segment']==s, 'monetary'] for s in cust_full['segment'].unique()]
f_stat, p_anova = stats.f_oneway(*groups)
print('ANOVA (monetary by segment): F=%.3f, p=%.6f' % (f_stat, p_anova))


In [ ]:

# ## 4) Classification – Churn
threshold = cust_full['recency_days'].quantile(0.75)
cust_full['churned'] = (cust_full['recency_days'] >= threshold).astype(int)

X = cust_full[['frequency','monetary','discount','avg_age','is_student','has_membership','country','region','segment','primary_channel']]
y = cust_full['churned']

num_features = ['frequency','monetary','discount','avg_age']
cat_features = ['country','region','segment','primary_channel']

preprocess = ColumnTransformer([
    ('num', StandardScaler(), num_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
], remainder='drop')

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

clf = Pipeline([('pre', preprocess), ('model', GradientBoostingClassifier())])
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
proba = clf.predict_proba(X_test)[:,1]

print('Accuracy:', accuracy_score(y_test, pred))
print('ROC-AUC:', roc_auc_score(y_test, proba))
print('\\nClassification Report:\\n', classification_report(y_test, pred))


In [ ]:

# ## 5) Regression – CLV proxy
Xr = cust_full[['frequency','discount','avg_age','is_student','has_membership','country','region','segment','primary_channel']]
yr = cust_full['monetary']

preprocess_r = ColumnTransformer([
    ('num', StandardScaler(), ['frequency','discount','avg_age']),
    ('cat', OneHotEncoder(handle_unknown='ignore'), ['country','region','segment','primary_channel'])
], remainder='drop')

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import math

Xr_train, Xr_test, yr_train, yr_test = train_test_split(Xr, yr, test_size=0.25, random_state=42)
reg = Pipeline([('pre', preprocess_r), ('model', RandomForestRegressor(n_estimators=250, random_state=42))])
reg.fit(Xr_train, yr_train)
yr_pred = reg.predict(Xr_test)
rmse = math.sqrt(mean_squared_error(yr_test, yr_pred))
print('R^2:', r2_score(yr_test, yr_pred))
print('RMSE:', rmse)


In [ ]:

# ## 6) Clustering – KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import seaborn as sns
import matplotlib.pyplot as plt

cluster_base = cust_full[['frequency','monetary','discount','avg_age']].copy()
scaler = StandardScaler()
Z = scaler.fit_transform(cluster_base)

kmeans = KMeans(n_clusters=4, n_init=10, random_state=42)
labels = kmeans.fit_predict(Z)
cust_full['cluster'] = labels
print('Silhouette score:', silhouette_score(Z, labels))

plt.figure()
sns.scatterplot(x=cust_full['frequency'], y=cust_full['monetary'], hue=cust_full['cluster'])
plt.title('Customer Clusters (frequency vs monetary)')
plt.show()

cust_full.groupby('cluster')[['frequency','monetary','discount','avg_age']].mean().round(2)


In [ ]:

# ## 7) Time-Series Trend
monthly = df.set_index('order_date')['revenue'].resample('M').sum()
ma3 = monthly.rolling(3).mean()

plt.figure()
monthly.plot(label='Monthly Revenue')
ma3.plot(label='3M Moving Average')
plt.title('Monthly Revenue with 3M Moving Average')
plt.legend()
plt.show()

monthly.tail(6)


In [ ]:

# ## 8) Insights & Recommendations
top_cats = df.groupby('category')['revenue'].sum().sort_values(ascending=False).head(5)
print('Top categories by revenue:\\n', top_cats)

avg_discount = df['discount'].mean()
print('\\nAverage discount rate overall:', round(avg_discount,3))

threshold = cust_full['recency_days'].quantile(0.75)
churn_rate = (cust_full['recency_days'] >= threshold).mean()
print('Estimated churn rate:', round(churn_rate,3))

print(\"\"\"\\nRecommendations:
1) Promote top categories through bundles and <=10% promos.
2) Launch win-back campaigns for high-recency segments; focus on low-frequency cohorts.
3) Membership drives for non-members; members spend more on average (validated via t-test).
4) Optimize digital channels (Web/Mobile) with A/B tests.
5) Monitor seasonality (Nov–Dec) and prepare inventory and marketing accordingly.
\"\"\")

In [ ]:

# ## 9) Wrap-up
print("Saving engineered customer features...")
cust_full.to_csv('/mnt/data/capstone_customer_features.csv', index=False)
"Capstone complete."
